In [0]:
from pyspark.sql.functions import col, struct, explode, collect_list,lit,array,split, expr
from pyspark.sql.types import StructType, StructField, StringType, ArrayType,FloatType,DoubleType,IntegerType,TimestampType
from time import sleep
from pyspark.sql.functions import udf
from pyspark.sql import DataFrame,SparkSession
import pyspark.sql.functions as F 
from pyspark.sql.functions import current_timestamp
import datetime

In [0]:
%sql
CREATE OR REPLACE TABLE iws_model AS
SELECT "root" AS node_id, "-" AS label, array("A") AS children_labels, array("root-A") AS children_id, 
    array(
        struct("root-A" AS node_id, "A" AS label, 1 as level, array() as events_between),
        struct("root-A-B" AS node_id, "B" AS label, 2 as level, array("A") as events_between),
        struct("root-A-B-X" AS node_id, "X" AS label, 3 as level, array("A","B") as events_between), 
        struct("root-A-B-D" AS node_id, "D" AS label, 3 as level, array("A","B") as events_between)
    ) AS nth_children, 0 as level
UNION ALL 
SELECT "root-A", "A", array("B"), array("root-A-B"), 
    array(
        struct("root-A-B" AS node_id, "B" AS label, 2 as level, array() as events_between),
        struct("root-A-B-X" AS node_id, "X" AS label, 3 as level, array("B") as events_between),
        struct("root-A-B-D" AS node_id, "D" AS label, 3 as level, array("B") as events_between),
        struct("root-A-B-D-C" AS node_id, "C" AS label, 4 as level, array("B","D") as events_between)
    ),1
UNION ALL 
SELECT "root-A-B", "B", array("X", "D"), array("root-A-B-X", "root-A-B-D"), 
    array(
        struct("root-A-B-X" AS node_id, "X" AS label, 3 as level, array() as events_between), -- "A-B-C" now "A-B-X"
        struct("root-A-B-D" AS node_id, "D" AS label, 3 as level, array() as events_between),
        struct("root-A-B-D-C" AS node_id, "C" AS label, 4 as level, array("D") as events_between)
    ),2
UNION ALL 
SELECT "root-A-B-X", "X", array(), array(),
    array(),3
UNION ALL 
SELECT "root-A-B-D", "D", array("C"), array("root-A-B-D-C"), 
    array(
        struct("root-A-B-D-C" AS node_id, "C" AS label, 4 as level, array() as events_between),
        struct("root-A-B-D-C-D" AS node_id, "D" AS label, 5 as level, array("C") as events_between),
        struct("root-A-B-D-C-C" AS node_id, "C" AS label, 5 as level, array("C") as events_between)
    ),3
UNION ALL 
SELECT "root-A-B-D-C", "C", array("D", "C"), array("root-A-B-D-C-D", "root-A-B-D-C-C"), 
    array(
        struct("root-A-B-D-C-D" AS node_id, "D" AS label, 5 as level, array() as events_between),
        struct("root-A-B-D-C-C" AS node_id, "C" AS label, 5 as level, array() as events_between)
    ),4
UNION ALL 
SELECT "root-A-B-D-C-D", "D", array(), array(), array(),5
UNION ALL 
SELECT "root-A-B-D-C-C", "C", array(), array(), array(),5;




num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE iws_event 
(event STRING,time_stamp TIMESTAMP,trace_id STRING);
---SELECT "A" event, CURRENT_TIMESTAMP() time_stamp, "trace_id_0" trace_id;


CREATE OR REPLACE TABLE iws_event_state
(event STRING, time_stamp TIMESTAMP, trace_id STRING,processed STRING);
CREATE OR REPLACE TABLE iws_labels
SELECT "A" AS event, "A"  AS label
UNION ALL
SELECT "B","B" 
UNION ALL
SELECT "C","C" 
UNION ALL
SELECT "D","D" 
UNION ALL
SELECT "X","X" ;

CREATE OR REPLACE TABLE iws_state
(trace_id STRING, ts TIMESTAMP, current_node STRING,current_id STRING,cost_of_alignment INTEGER,previous_events STRING, trace STRING, execution_sequence STRING,event_level
INTEGER,current_node_level INTEGER);
---event level to filter out the latest alignments later

In [0]:
import random 

activities = ["A","B","C","D","X"]
trace_id_lower = 0
trace_id_upper = 9

def insert_event(event=None, trace_id=None):
    if not event:
        event = random.choice(activities)
    if not trace_id:
        trace_id = 0 #random.randint(trace_id_lower, trace_id_upper)
    spark.sql(f"INSERT INTO iws_event SELECT '{event}', CURRENT_TIMESTAMP(), 'trace_id_{trace_id}'")

In [0]:
event_df = spark.readStream.table("iws_event").withWatermark("time_stamp", "1 minute")
event_df.createOrReplaceTempView("events")

In [0]:
%scala
import scala.collection.mutable.ArrayBuffer

def calculateAlignmentCost(modelEvents: String, eventArray: Array[String]): Array[(String, String, Int, Int)] = {
  val newEvents = modelEvents.replace("-", "").split("")
  val n = eventArray.length
  val m = newEvents.length
  val dp = Array.tabulate(n + 1, m + 1)((i, j) => if (i == 0) j else if (j == 0) i else 0)

  if (modelEvents == "") {
    return eventArray.zipWithIndex.map { case (event, index) =>
      (event, "log", 1, index + 1)
    }
  }

  // Fill the matrix
  for (i <- 1 to n) {
    for (j <- 1 to m) {
      if (eventArray(i - 1) == newEvents(j - 1)) {
        dp(i)(j) = dp(i - 1)(j - 1)
      } else {
        dp(i)(j) = math.min(dp(i - 1)(j) + 1, dp(i)(j - 1) + 1)
      }
    }
  }

  // Track back to build the alignment
  val alignment = ArrayBuffer[(String, String, Int, Int)]()
  var i = n
  var j = m
  var cost = dp(n)(m)

  while (i > 0 && j > 0) {
    if (eventArray(i - 1) == newEvents(j - 1)) {
      alignment.prepend((eventArray(i - 1), "sync", 0, i))
      i -= 1
      j -= 1
    } else if (dp(i)(j) == dp(i - 1)(j) + 1) {
      alignment.prepend((eventArray(i - 1), "log", 1, i))
      i -= 1
    } else {
      alignment.prepend((newEvents(j - 1), "model", 1, j))
      j -= 1
    }
  }
  // Handle any remaining elements necessary if there are trailing events in the beginning of the trace or model nodes
  while (i > 0) {
    alignment.prepend((eventArray(i - 1), "log", 1, i))
    i -= 1
  }
  while (j > 0) {
    alignment.prepend((newEvents(j - 1), "model", 1, j))
    j -= 1
  }


  alignment.toArray
}

// Registering the UDF
spark.udf.register("calculateAlignmentCost", (modelEvents: String, eventArray: Array[String]) => calculateAlignmentCost(modelEvents, eventArray))


import scala.collection.mutable.ArrayBuffer
calculateAlignmentCost: (modelEvents: String, eventArray: Array[String])Array[(String, String, Int, Int)]
res0: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$10696/1567368326@793a9708,ArrayType(StructType(StructField(_1,StringType,true),StructField(_2,StringType,true),StructField(_3,IntegerType,false),StructField(_4,IntegerType,false)),true),List(Some(class[value[0]: string]), Some(class[value[0]: array<string>])),Some(class[value[0]: array<struct<_1:string,_2:string,_3:int,_4:int>>]),Some(calculateAlignmentCost),true,true)

In [0]:
%sql
CREATE OR REPLACE TABLE state_test_comp
(trace_id STRING, event_array ARRAY<STRUCT<time_stamp TIMESTAMP,label STRING>>,len INTEGER,previous_id STRING,cost_of_alignment INTEGER,previous_events STRING,previous_alignment ARRAY<STRUCT<event String,move_type String>>,event_level INTEGER,current_event_level INTEGER,current_node_level INTEGER);

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW POSSIBLE_ALIGNMENTS AS (
SELECT trace_id, node_id as current_id ,cost_of_alignment + aggregate(calc_alignment._3, 0, (acc, x) -> acc + x) AS cost_of_alignment,concat(previous_events,concat_ws("",current_events)) as previous_events,CONCAT(
                    previous_alignment,
                    TRANSFORM(
                        calc_alignment,
                            x -> named_struct('event', x._1, 'move_type', x._2)
                        )
                 ) AS alignment,current_event_level + size(current_events) as event_level,event_index,time_stamp,incoming_label,level as current_node_level,current_events FROM (
SELECT *,calculateAlignmentCost(substr(node_id FROM len(previous_id) + 1),current_events) as calc_alignment 
FROM (SELECT DISTINCT *, idx AS event_index, 
                col.time_stamp as time_stamp,
                col.label as incoming_label,
                slice(event_array,current_event_level+1,len - current_event_level).label as current_events  
                from state_test_comp
                LATERAL VIEW posexplode(slice(event_array,current_event_level+1,len-current_event_level)) AS idx, col) f
                JOIN iws_model m ON (m.node_id LIKE CONCAT(f.previous_id, '%') AND
                m.level < f.current_node_level + event_index + 3 + 2 AND
                m.label = f.incoming_label) OR
                m.node_id = f.previous_id) );

CREATE OR REPLACE temp VIEW stream_test_alignm AS SELECT DISTINCT 
                trace_id,
                current_id,
                previous_events,
                alignment,
                cost_of_alignment,
                event_level,
                max_event_level as current_event_level,
                current_node_level
FROM   (
        SELECT *,
        Max(event_level) OVER (partition BY trace_id) AS max_event_level,
        Row_number() OVER (partition BY trace_id,current_id ORDER BY event_level DESC,cost_of_alignment ASC, Len(previous_events) DESC) rn
        FROM POSSIBLE_ALIGNMENTS)
WHERE rn = 1

In [0]:
spark.sql("""SELECT
            e.trace_id as trace_id,
            e.event_array as event_array,
            size(e.event_array) as len,
            COALESCE(r.current_id, 'root') AS previous_id,
            COALESCE(r.cost_of_alignment, 0) AS cost_of_alignment,
            COALESCE(r.previous_events, '') AS previous_events,
            COALESCE(r.alignment, ARRAY(struct("" AS event, "" AS move_type))) AS previous_alignment,
            COALESCE(r.event_level, 0) AS event_level,
            COALESCE(r.current_event_level, 0) AS current_event_level,
            COALESCE(r.current_node_level, 0) AS current_node_level
            FROM (
                SELECT trace_id,
                        array_sort(collect_list(struct(time_stamp, label))) AS event_array
                    FROM 
                        events e
                    JOIN 
                        iws_labels l ON e.event = l.event
                    GROUP BY trace_id) e
            LEFT JOIN stream_test_alignm r ON e.trace_id = r.trace_id
""").writeStream.format("delta").outputMode("complete").option("checkpointLocation", "/tmp/delta/state_append_20022/").toTable("state_test_comp")


Out[12]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fa1350ef220>

In [0]:
insert_event("C",3)
#insert different events with different traces
#At the moment inserted events are AXBAC
#AXBA

In [0]:
%sql
select * from POSSIBLE_ALIGNMENTS where trace_id = "trace_id_3"

trace_id,current_id,cost_of_alignment,previous_events,alignment,event_level,event_index,time_stamp,incoming_label,current_node_level,current_events
trace_id_3,root-A-B-D-C,5,AXBAC,"List(List(, ), List(A, log), List(X, log), List(B, log), List(A, sync), List(B, model), List(D, model), List(C, sync))",5,0,2024-06-06T19:00:46.287+0000,C,4,List(C)
trace_id_3,root-A-B-D-C,7,AXBAC,"List(List(, ), List(A, log), List(X, log), List(B, log), List(A, log), List(A, model), List(B, model), List(D, model), List(C, sync))",5,0,2024-06-06T19:00:46.287+0000,C,4,List(C)
trace_id_3,root-A-B-D-C,3,AXBAC,"List(List(, ), List(A, sync), List(X, log), List(B, sync), List(A, log), List(D, model), List(C, sync))",5,0,2024-06-06T19:00:46.287+0000,C,4,List(C)
trace_id_3,root-A-B,3,AXBAC,"List(List(, ), List(A, sync), List(X, log), List(B, sync), List(A, log), List(C, log))",5,0,2024-06-06T19:00:46.287+0000,C,2,List(C)
trace_id_3,root-A,4,AXBAC,"List(List(, ), List(A, log), List(X, log), List(B, log), List(A, sync), List(C, log))",5,0,2024-06-06T19:00:46.287+0000,C,1,List(C)
trace_id_3,root,5,AXBAC,"List(List(, ), List(A, log), List(X, log), List(B, log), List(A, log), List(C, log))",5,0,2024-06-06T19:00:46.287+0000,C,0,List(C)
trace_id_3,root-A-B-D-C-C,6,AXBAC,"List(List(, ), List(A, log), List(X, log), List(B, log), List(A, sync), List(B, model), List(D, model), List(C, model), List(C, sync))",5,0,2024-06-06T19:00:46.287+0000,C,5,List(C)
trace_id_3,root-A-B-D-C-C,4,AXBAC,"List(List(, ), List(A, sync), List(X, log), List(B, sync), List(A, log), List(D, model), List(C, model), List(C, sync))",5,0,2024-06-06T19:00:46.287+0000,C,5,List(C)
trace_id_3,root-A-B-X,4,AXBAC,"List(List(, ), List(A, sync), List(B, model), List(X, sync), List(B, log), List(A, log), List(C, log))",5,0,2024-06-06T19:00:46.287+0000,C,3,List(C)


In [0]:
%sql
SELECT trace_id,current_id,previous_events,slice(alignment, 2, size(alignment)),cost_of_alignment
FROM POSSIBLE_ALIGNMENTS 
WHERE (trace_id, cost_of_alignment) IN (
    SELECT trace_id, MIN(cost_of_alignment)
    FROM POSSIBLE_ALIGNMENTS 
    GROUP BY trace_id
) and trace_id = "trace_id_3";
--- select the final optimal prefix alignments

trace_id,current_id,previous_events,"slice(alignment, 2, size(alignment))",cost_of_alignment
trace_id_3,root-A-B-D-C,AXBAC,"List(List(A, sync), List(X, log), List(B, sync), List(A, log), List(D, model), List(C, sync))",3
trace_id_3,root-A-B,AXBAC,"List(List(A, sync), List(X, log), List(B, sync), List(A, log), List(C, log))",3
